In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, HuberRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import StandardScaler, QuantileTransformer, PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.inspection import permutation_importance

from sklearn.model_selection import GridSearchCV

from scipy.stats import kurtosis, skew

In [ ]:
def get_metrics(y_true, y_pred, model=None, weights=None):
    if model:
        print(f'Model {model}')
        
    print(f'  MAE = {mean_absolute_error(y_true, y_pred, sample_weight=weights):.3f}')
    print(f'  MSE = {mean_squared_error(y_true, y_pred, sample_weight=weights):.3f}')
    print(f'  Max error = {max_error(y_true, y_pred):.3f}')
    print(f'  R2 = {r2_score(y_true, y_pred, sample_weight=weights):.3f}')

In [ ]:
# for grid search cv
def cv_splitter(X):
    for i in range(5):
        start_ind = int(len(X)/10 * i)
        split_ind = int(len(X)/10 * (i + 5))
        end_ind = int(len(X)/10 * (i + 6))
        train_ind = np.arange(start_ind, split_ind)
        test_ind = np.arange(split_ind, end_ind)
        yield train_ind, test_ind

In [ ]:
def get_split(df, to_drop=['TRADEDATE', 'SECID', 'VALUE', 'CLOSE', 'FACEUNIT', 'FACEVALUE', 'ISSUESIZE',
                           'HIGH', 'LOW', 'delta_1', 'max_delta', 'delta_2', 'delta_3', 'delta_4', 'delta_5', 'delta_6',
                            'delta_7', 'delta_8', 'delta_9', 'delta_10', 'TOMAT'
                           ], trans=None):
    """
    Returns: X_train, X_test, y_train, y_test
    """
    
    X = pd.get_dummies(df.drop(to_drop, axis=1), columns = ['TYPE', 'COUPON_TYPE'])
    
    if trans:
        X_train = trans.fit_transform(X[:int(len(df)*3/4)])
        X_test = trans.transform(X[int(len(df)*3/4):])
        
    else:
        X_train = X[:int(len(df)*3/4)]
        X_test = X[int(len(df)*3/4):]
        
    y_train = df['CLOSE'][:int(len(df)*3/4)]
    y_test = df['CLOSE'][int(len(df)*3/4):]
    return X_train, X_test, y_train, y_test

In [ ]:
def mape(y_true, y_pred, weights):
    mape = np.abs(y_true-y_pred)/np.abs(y_true)
    return np.average(mape, weights=weights)

In [ ]:
def full_cv(df, model, start_date, folder, scale_x=None, scale_y=None):
    
    date_str = str(start_date).split('T')[0]
    df_today = df[df['TRADEDATE'] == start_date]
    df_test = df_today.sample(int(len(df_today)/4), weights=df_today['WEIGHT'], random_state=6)
    df_train = df[df['TRADEDATE'] <= start_date].drop(df_test.index)
    
    if scale_x:
        X_train = scale_x.fit_transform(df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'],
                                                      axis=1).drop([f'delta{i}' for i in range(1,11)], axis=1))
        X_test = scale_x.transform(df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                axis=1).drop([f'delta_{i}' for i in range(1,11)], axis=1))
        X_train_return = scale_x.fit_transform(df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                             axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1))
        X_test_return = scale_x.transform(df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                             axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1))
    else:
        X_train = df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], axis=1)
        X_test = df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], axis=1)
        
    if scale_y:
        y_train_delta = scale_y.fit_transform(df_train['DELTA'].values.reshape(-1,1)).reshape(-1,)
        y_train_return = scale_y.fit_transform(df_train['RETURN'].values.reshape(-1,1)).reshape(-1,)
        y_train_close = scale_y.fit_transform(df_train['CLOSE'].values.reshape(-1,1)).reshape(-1,)
        y_test_close = scale_y.transform(df_test['CLOSE'].values.reshape(-1,1)).reshape(-1,)
    else:
        y_train_close = df_train['CLOSE']
        y_test_close = df_test['CLOSE']
        y_train_delta = df_train['DELTA']
        y_train_return = df_train['RETURN']
        
    model.fit(X_train, y_train_close)
    pred = model.predict(X_test)
    pd.Series(pred).to_csv(f'{folder}/{date_str}_close.csv', index=False)
    qual_close = mape(y_test_close, pred, df_test['WEIGHT'])
    
    model.fit(X_train, y_train_delta)
    pred = model.predict(X_test) + X_test['CLOSE_1']
    pd.Series(pred).to_csv(f'{folder}/{date_str}_delta.csv', index=False)
    qual_delta = mape(y_test_close, pred, df_test['WEIGHT'])
    
    model.fit(X_train, y_train_return)
    pred = model.predict(X_test) * X_test['CLOSE_1'] + X_test['CLOSE_1']
    pd.Series(pred).to_csv(f'{folder}/{date_str}_return.csv', index=False)
    qual_return = mape(y_test_close, pred, df_test['WEIGHT'])
    
    return qual_close, qual_delta, qual_return

In [ ]:
def feature_importance(model, X_test, y_test, weights, close, target='CLOSE'):
    imps = []
    
    if target == 'CLOSE':
        mape_full = mape(model.predict(X_test), y_test, weights)
        for i in range(X_test.shape[1]):
            mape_curr = mape(model.predict(np.hstack((X_test.values[:, :i], np.zeros((len(X_test), 1)), 
                                                     X_test.values[:, i+1:]))), 
                             y_test, weights)
            imps.append(mape_curr - mape_full)
            
    elif target == 'DELTA':
        mape_full = mape(model.predict(X_test) + close, y_test, weights)
        for i in range(X_test.shape[1]):
            mape_curr = mape(model.predict(np.hstack((X_test.values[:, :i], np.zeros((len(X_test), 1)), 
                                                     X_test.values[:, i+1:]))) + close, 
                             y_test, weights)
            imps.append(mape_curr - mape_full)
    
    elif target == 'RETURN':
        mape_full = mape(model.predict(X_test) * close + close, y_test, weights)
        for i in range(X_test.shape[1]):
            mape_curr = mape(model.predict(np.hstack((X_test.values[:, :i], np.zeros((len(X_test),1)), 
                                                     X_test.values[:, i+1:]))) * close + close, 
                             y_test, weights)
            imps.append(mape_curr - mape_full)
            
    return imps

In [ ]:
def cross_validate(df, model_price, model_return, folder, scale_x_price=None, scale_x_return=None, scale_y=None):
    
    quals_close = []
    quals_delta = []
    quals_return = []
    imps_close = []
    imps_delta = []
    imps_return = []
    
    for start_ind, split_ind, end_ind in tqdm([(int(len(df[:381251])/10*i), int(len(df[:381251])/10*(i+5)), 
                                           int(len(df[:381251])/10 * (i + 6))) for i in range(5)]):
        
    
        df_train = df[start_ind:split_ind]
        df_test = df[split_ind:end_ind]
        
        X_train = df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                axis=1).drop([f'delta_{i}' for i in range(2,11)], axis=1)
        
        X_test = df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'],
                              axis=1).drop([f'delta_{i}' for i in range(2,11)], axis=1)

        X_train_return = df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                          axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1)
        
        X_test_return = df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                          axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1)

        if scale_y:
            y_train_close = scale_y.fit_transform(df_train['CLOSE'].values.reshape(-1,1)).reshape(-1,)
            y_test_close = scale_y.transform(df_test['CLOSE'].values.reshape(-1,1)).reshape(-1,)
            scale_close = scale_y
            y_train_delta = scale_y.fit_transform(df_train['DELTA'].values.reshape(-1,1)).reshape(-1,)
            y_test_delta = scale_y.transform(df_test['DELTA'].values.reshape(-1,1)).reshape(-1,)
            scale_delta = scale_y
            y_train_return = scale_y.fit_transform(df_train['RETURN'].values.reshape(-1,1)).reshape(-1,)
            y_test_return = scale_y.transform(df_test['RETURN'].values.reshape(-1,1)).reshape(-1,)
            scale_return = scale_y
            test = df_test['CLOSE']

        else:
            y_train_close = df_train['CLOSE'].values
            y_test_close = df_test['CLOSE'].values
            y_train_delta = df_train['DELTA'].values
            y_test_delta = df_test['DELTA'].values
            y_train_return = df_train['RETURN'].values
            y_test_return = df_test['RETURN'].values
            

        model_price.fit(X_train, y_train_close, model__sample_weight=df_train['WEIGHT'])
        
        if scale_y:
            pred = scale_close.inverse_transform(model_price.predict(X_test))
            pd.Series(pred).to_csv(f'{folder}/{start_ind}_close.csv', index=False)
            qual_close = [mape(test, pred, df_test['WEIGHT']), 
                      mean_absolute_error(test, pred, sample_weight=df_test['WEIGHT'])]
        else:
            pred = model_price.predict(X_test)
            pd.Series(pred).to_csv(f'{folder}/{start_ind}_close.csv', index=False)
            qual_close = [mape(y_test_close, pred, df_test['WEIGHT']), 
                          mean_absolute_error(y_test_close, pred, sample_weight=df_test['WEIGHT'])]
            
        quals_close.append(qual_close)
        imps_close.append(feature_importance(model_price, X_test, y_test_close, df_test['WEIGHT'], X_test['CLOSE_1'],
                                             target='CLOSE'))

        model_price.fit(X_train, y_train_delta, model__sample_weight=df_train['WEIGHT'])
        
        if scale_y:
                pred = scale_delta.inverse_transform(model_price.predict(X_test)) + df_test['CLOSE_1']
                pd.Series(pred).to_csv(f'{folder}/{start_ind}_delta.csv', index=False)
                qual_delta = [mape(test, pred, df_test['WEIGHT']),
                      mean_absolute_error(test, pred, sample_weight=df_test['WEIGHT'])]
        else:                
            pred = model_price.predict(X_test) + df_test['CLOSE_1']
            pd.Series(pred).to_csv(f'{folder}/{start_ind}_delta.csv', index=False)
            qual_delta = [mape(y_test_close, pred, df_test['WEIGHT']),
                          mean_absolute_error(y_test_close, pred, sample_weight=df_test['WEIGHT'])]
            
        quals_delta.append(qual_delta)
        imps_delta.append(feature_importance(model_price, X_test, y_test_delta, df_test['WEIGHT'], X_test['CLOSE_1'],
                                             target='DELTA'))

        model_return.fit(X_train_return, y_train_return, model__sample_weight=df_train['WEIGHT'])
        
        if scale_y:
            pred = scale_return.inverse_transform(model_return.predict(X_test_return)) * df_test['CLOSE_1'] + df_test['CLOSE_1']
            pd.Series(pred).to_csv(f'{folder}/{start_ind}_return.csv', index=False)
            qual_return = [mape(test, pred, df_test['WEIGHT']),
                       mean_absolute_error(test, pred, sample_weight=df_test['WEIGHT'])]
        else:
            pred = model_return.predict(X_test_return) * df_test['CLOSE_1'] + df_test['CLOSE_1']
            pd.Series(pred).to_csv(f'{folder}/{start_ind}_return.csv', index=False)
            qual_return = [mape(y_test_close, pred, df_test['WEIGHT']),
                           mean_absolute_error(y_test_close, pred, sample_weight=df_test['WEIGHT'])]
            
        quals_return.append(qual_return)
        imps_return.append(feature_importance(model_return, X_test_return, y_test_return, df_test['WEIGHT'], 
                                              X_test['CLOSE_1'], target='RETURN'))

    return quals_close, quals_delta, quals_return, imps_close, imps_delta, imps_return

In [ ]:
def predict_full(df, model_price, model_return, folder, weights=True):
    
    df_train = df[:381251]
    df_test = df[381251:]
    
    X_train = df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                axis=1).drop([f'delta_{i}' for i in range(2,11)], axis=1)
        
    X_test = df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'],
                              axis=1).drop([f'delta_{i}' for i in range(2,11)], axis=1)

    X_train_return = df_train.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                          axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1)
        
    X_test_return = df_test.drop(['CLOSE', 'DELTA', 'RETURN', 'TRADEDATE', 'WEIGHT'], 
                                                          axis=1).drop([f'CLOSE_{i}' for i in range(1,11)], axis=1)
    
    y_train_close = df_train['CLOSE'].values
    y_test_close = df_test['CLOSE'].values
    y_train_delta = df_train['DELTA'].values
    y_test_delta = df_test['DELTA'].values
    y_train_return = df_train['RETURN'].values
    y_test_return = df_test['RETURN'].values
    
    if weights:
        model_price.fit(X_train, y_train_close, model__sample_weight=df_train['WEIGHT'])
    else:
        model_price.fit(X_train, y_train_close)
        
    pred = model_price.predict(X_test)
    pd.Series(pred).to_csv(f'{folder}/full_close.csv', index=False)
    print(f'{folder} Close done')
    
    if weights:
        model_price.fit(X_train, y_train_delta, model__sample_weight=df_train['WEIGHT'])
    else:
         model_price.fit(X_train, y_train_delta)
            
    pred = model_price.predict(X_test) + df_test['CLOSE_1']
    pd.Series(pred).to_csv(f'{folder}/full_delta.csv', index=False)
    print(f'{folder} Delta done')
    
    if weights:
        model_return.fit(X_train_return, y_train_return, model__sample_weight=df_train['WEIGHT'])
    else:
        model_return.fit(X_train_return, y_train_return)
        
    pred = model_return.predict(X_test_return) * df_test['CLOSE_1'] + df_test['CLOSE_1']
    pd.Series(pred).to_csv(f'{folder}/full_return.csv', index=False)
    print(f'{folder} Return done')

In [ ]:
lr = Pipeline([('model', LinearRegression())])
ridge_close = Pipeline([('scaler', MinMaxScaler()), ('model', Ridge(alpha=0.01, max_iter=5000))])
ridge_return = Pipeline([('scaler', MinMaxScaler()), ('model', Ridge(alpha=10,  max_iter=5000))])
lasso_close = Pipeline([('scaler', StandardScaler()),('model', Lasso(alpha=0.01,  max_iter=5000))])
lasso_return = Pipeline([('scaler', StandardScaler()),('model', Lasso(alpha=0.001,  max_iter=5000))])
huber_close = Pipeline([('scaler', StandardScaler()), ('model', HuberRegressor(alpha=0.001, epsilon=1.01, max_iter=5000))])
huber_return = Pipeline([('scaler', MinMaxScaler()), ('model', HuberRegressor(alpha=1, epsilon=1.01,  max_iter=5000))])
elastic_close = Pipeline([('scaler', StandardScaler()), ('model', ElasticNet(alpha=0.01, l1_ratio=0.9,  max_iter=5000))])
elastic_return = Pipeline([('scaler', StandardScaler()), ('model', ElasticNet(alpha=0.001, l1_ratio=0.6,  max_iter=5000))])
rf = Pipeline([('model', RandomForestRegressor(random_state=6, ccp_alpha=0.005, max_depth=30, n_estimators=100))])

for model_close, model_return, folder in [(lr, lr, 'lr_pred'), (ridge_close, ridge_return, 'ridge_pred'),
                                 (lasso_close, lasso_return, 'lasso_pred'), (huber_close, huber_return, 'huber_pred'),
                                 (elastic_close, elastic_return, 'elastic_pred'), (rf, rf, 'rf_pred')]:
    predict_full(df_clean, model_close, model_return, folder)

In [ ]:
adaboost_close = Pipeline([('model', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=7, random_state=6),
                                                      n_estimators=100, learning_rate=0.001))])
adaboost_return = Pipeline([('model', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=6, random_state=6),
                                                      n_estimators=100, learning_rate=0.01))])
gradboost_close = Pipeline([('model', GradientBoostingRegressor(max_depth=6, loss='huber', n_estimators=100))])
gradboost_close = Pipeline([('model', GradientBoostingRegressor(max_depth=7, loss='huber', n_estimators=50))])

In [ ]:
%%time
predict_full(df_clean, adaboost_close, adaboost_return, 'Predictions/adaboost_pred', weights=False)

In [ ]:
%%time
predict_full(df_clean, gradboost_close, gradboost_close, 'Predictions/gradboost_pred', weights=True)

# Return prediction

In [ ]:
df_for_return = df.copy()

In [ ]:
df_for_return['RETURN'] = (df_for_return['CLOSE'] - df_for_return['CLOSE_1'])/df_for_return['CLOSE_1']

In [ ]:
df_for_return['delta_1'] = (df_for_return['CLOSE'] - df_for_return['CLOSE_1'])/df_for_return['CLOSE_1']
df_for_return['delta_2'] = (df_for_return['CLOSE_1'] - df_for_return['CLOSE_2'])/df_for_return['CLOSE_2']
df_for_return['delta_3'] = (df_for_return['CLOSE_2'] - df_for_return['CLOSE_3'])/df_for_return['CLOSE_3']
df_for_return['delta_4'] = (df_for_return['CLOSE_3'] - df_for_return['CLOSE_4'])/df_for_return['CLOSE_4']
df_for_return['delta_5'] = (df_for_return['CLOSE_4'] - df_for_return['CLOSE_5'])/df_for_return['CLOSE_5']
df_for_return['delta_6'] = (df_for_return['CLOSE_5'] - df_for_return['CLOSE_6'])/df_for_return['CLOSE_6']
df_for_return['delta_7'] = (df_for_return['CLOSE_6'] - df_for_return['CLOSE_7'])/df_for_return['CLOSE_7']
df_for_return['delta_8'] = (df_for_return['CLOSE_7'] - df_for_return['CLOSE_8'])/df_for_return['CLOSE_8']
df_for_return['delta_9'] = (df_for_return['CLOSE_8'] - df_for_return['CLOSE_9'])/df_for_return['CLOSE_9']
df_for_return['delta_10'] = (df_for_return['CLOSE_9'] - df_for_return['CLOSE_10'])/df_for_return['CLOSE_10']

df_for_return['max_delta'] = df_for_return[[f'delta_{i}' for i in range(1, 11)]].abs().max(axis=1)

df_for_return_clean = df_for_return[df_for_return['max_delta'].abs() < 10].copy()

In [ ]:
def get_split_return(df, to_drop=['TRADEDATE', 'SECID', 'VALUE', 'FACEUNIT', 'FACEVALUE', 'ISSUESIZE',
                           'HIGH', 'LOW', 'TOMAT'
                           ], trans=None):
    """
    Returns: X_train, X_test, y_train, y_test
    """
    
    X = pd.get_dummies(df.drop(to_drop, axis=1), columns = ['TYPE', 'COUPON_TYPE'])
    
    if trans:
        X_train = trans.fit_transform(X[:int(len(df)*3/4)])
        X_test = trans.transform(X[int(len(df)*3/4):])
        
    else:
        X_train = X.drop('RETURN', axis=1)[:int(len(df)*3/4)]
        X_test = X.drop('RETURN', axis=1)[int(len(df)*3/4):]
        
    y_train = df['RETURN'][:int(len(df)*3/4)]
    y_test = df['RETURN'][int(len(df)*3/4):]
    return X_train, X_test, y_train, y_test

In [ ]:
df_for_return_clean = df_for_return_clean.drop(
    [f'CLOSE_{i}' for i in range(1, 11)], axis=1).drop(['CLOSE', 'max_delta', 'delta_1', 'WEIGHT'], axis=1)

df_for_return_clean[[f'DIST_{i}' for i in range(1, 11)]] =\
np.log(1 + df_for_return_clean[[f'DIST_{i}' for i in range(1, 11)]])

df_for_return_clean[[f'VALUE_{i}' for i in range(1, 11)]] =\
np.log(1 + df_for_return_clean[[f'VALUE_{i}' for i in range(1, 11)]])

In [ ]:
X_train, X_test, y_train, y_test = get_split_return(df_for_return_clean)

In [ ]:
lr = LinearRegression()
huber = HuberRegressor()
rf = RandomForestRegressor(random_state=6, criterion='mae', n_jobs=2, max_depth=20)
gb = GradientBoostingRegressor(loss='lad', random_state=6)

In [ ]:
lr.fit(X_train, y_train)
get_metrics(y_test, lr.predict(X_test), model='RETURN')
get_metrics(df['CLOSE'][X_test.index], lr.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE')
get_metrics(df['CLOSE'][X_test.index], lr.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE_WEIGHT', weights=df['WEIGHT'][X_test.index])

In [ ]:
huber.fit(X_train, y_train)
get_metrics(y_test, huber.predict(X_test), model='RETURN')
get_metrics(df['CLOSE'][X_test.index], huber.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE')
get_metrics(df['CLOSE'][X_test.index], huber.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE_WEIGHT', weights=df['WEIGHT'][X_test.index])

In [ ]:
%%time
rf.fit(X_train, y_train)
get_metrics(y_test, rf.predict(X_test), model='RETURN')
get_metrics(df['CLOSE'][X_test.index], rf.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE')
get_metrics(df['CLOSE'][X_test.index], rf.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE_WEIGHT', weights=df['WEIGHT'][X_test.index])

In [ ]:
%%time
gb.fit(X_train, y_train)
get_metrics(y_test, gb.predict(X_test), model='RETURN')
get_metrics(df['CLOSE'][X_test.index], gb.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE')
get_metrics(df['CLOSE'][X_test.index], gb.predict(X_test) * df['CLOSE_1'][X_test.index] + df['CLOSE_1'][X_test.index],
           model='PRICE_WEIGHT', weights=df['WEIGHT'][X_test.index])